# Dynamic programming again: Implementing the Viterbi algorithm

As discussed in class, finite-state automata can be made probabilistic by adding a weight to every transition arc.
**Hidden Markov Models** (HMMs) provide an alternative format of specifying probabilistic FSAs.
This unit presents an implementation of arguably the Viterbi algorithm for HMMs.

## Background on HMMs

The details of HMMs are covered in the lecture notes, but a brief reminder never hurts:

1. The arcs between states are unlabeled, but have a probability that encodes how likely the automaton is to switch from one state to the other.
These are the **transition probabilities**.
1. In addition, each state has a table of **emission probabilities**, which tell us how likely it is to encounter a given symbol while the automaton is in this state.

In this notebook, we will also assume that there is exactly one initial state.
The initial state's emission probabilities are always 0.
It's only purpose, then, is to provide transition probabilities to the other states.
This gives us a basic starting point for the HMM.

In addition, we will also assume that there is exactly one final state and that one can always transition from any state into the final state to stop there.
This is just a technical trick that will make simplify some aspects of the implementation.

Given a HMM, one often wants to know what the most likely sequence of states is for a given input.
Just like the Levenshtein distance, this is a problem that cannot be solved with brute force.
Suppose our HMM has only two states N (for noun) and V (for verb).
Then a simple sentence like "police police police police police" already has `2 ** 5 = 32` different paths.
In some applications like genome sequencing, the HMM might have hundreds of states and the string might be thousands of symbols long.
Needless to say, `100 ** 1000` is way too big a number to compute the probability for each path from scratch.
Instead, we use dynamic programming to compute the probabilities for subpaths, store these values, and reuse them to compute the values of longer paths.

The rest of this notebook presupposes that you're already familiar with the Viterbi algorithm and its use of a table for computing probabilities.

## Basic HMM implementation

We start with a very simple class `hmm` for HMMs.
It largely builds on our `fsa` class.
Transmissions are passed in as a list of triples and then converted into a dictionary via `update`.
The format is slightly different, though.
The triples are assumed to follow the pattern `(source_state, goal_state, transition_probability)`.
Similarly, the dictionary has a slightly different format that is easier to use with the Viterbi algorithm:

```python
{source_state: {goal_state: transition_probablity}}
```

We also pass in emissions as triples of the form `(state, symbol, emission_probability)`.
The method `set_emit` is then used to construct a dictionary of the form

```python
{state: {symbol: emission_probability}}
```

We also add a variable `self.states` that keeps track of all the states that are not the initial state (by default `0`) or the final state (by default `-1`).
This, too, will simplify some aspects of the Viterbi algorithm.

Finally, we add a transition from every state to the final state.
For convenience, we sets its probability to 1.
This means that we do not discriminate between states as to whether they can occur at the very end of a state sequence.

In [ ]:
class hmm():
    def __init__(self, start=0, final=-1, trans=[], emissions=[]):
        self.start = start
        self.final = -1
        self.states = set()
        self.trans = {}
        for t in trans:
            self.update(t)
        self.emissions = {}
        for e in emissions:
            self.set_emit(e)
        for state in self.states:
            self.trans[state] = self.trans.get(state, {})
            self.trans[state][final] = 1
        
        
    def update(self, trans):
        source, goal, prob = trans
        self.trans[source] = self.trans.get(source, {})
        self.trans[source][goal] = prob
        # add new states to hmm's set of states
        if source != self.start:
            self.states.add(source)
        if goal != self.final:
            self.states.add(goal)
        
        
    def set_emit(self, e):
        state, word, prob = e
        self.emissions[state] = self.emissions.get(state, {})
        self.emissions[state][word] = prob

In [ ]:
from pprint import pprint

police = hmm(trans=[(0, "N", .8),
                    (0, "V", .2),
                    ("N", "N", .3),
                    ("N", "V", .7),
                    ("V", "N", .9),
                    ("V", "V", .1)],
            emissions=[("V", "police", 1),
                       ("N", "police", 1)])

print("Transition probabilities:")
pprint(police.trans, width=1)

print("Emission probabilities:")
pprint(police.emissions, width=1)

## Sketching out the Viterbi algorithm

The central tool of the Viterbi algorithm is the table in which it keeps track of probabilities.
The table contains a column for each position in the input, and a row for each state of the HMM (excluding the inital state and the final state).
The algorithm fills in the cells of this table column-by-column from left to right.
In order to compute the value of a cell, one needs easy access to all the cells of the previous column.

Whatever data structure we use to implement this table in Python, it needs to be something that gives us easy access to each column.
A natural choice is a dictionary where each key is a position in the string and the value is the corresponding column in the Viterbi table.
Each column, in term, is a dictionary where each key indicates a row, i.e. the value for a specific state.

For instance, the example HMM above would use the following dictionary for the input `["police", "police", "police"]`:

```python
{0: {"N": "value in column 0 for N",
     "V": "value in column 0 for V"}
 1: {"N": "value in column 1 for N",
     "V": "value in column 1 for V"},
 2: {"N": "value in column 2 for N",
     "V": "value in column 2 for V"}
}
```

Instead of directly adding the code to our class, let's first sketch a function that could construct such a table for us.

In [ ]:
def matrix(sentence, states):
    # initialize empty dictionary
    matrix = {}
    # construct matrix column by column
    for pos in range(len(sentence)):
        # initialize empty column
        matrix[pos] = {}
        # add cell for each row in the current column
        for state in states:
            matrix[pos][state] = "some_value"
    return matrix

In [ ]:
from pprint import pprint

pprint(matrix(["police", "police", "police"], ["N", "V"]), width=1)

Alright, this looks pretty much like what we had in mind.
The next step is to figure out what we should put in those cells.
Remember that the Viterbi algorithm actually stores two values in each cell:

1. the highest possible probability value, and
1. a backpointer to a cell in the previous column that the probability value was obtained from.

In rare occassions there might actually be multiple cells in the previous column that can produce the highest probability value.
For simplicity, we will ignore this case for now.
Either way, though, we need to store two values per cell.
The simplest option is, once again, a dictionary, with a key `"prob"` for the probability and a key `"from"` for the backpointer.
The value of `"from"` is the state that represents the row of the cell that the backpointer picks out.

In [ ]:
def matrix(sentence, states):
    matrix = {}
    for pos in range(len(sentence)):
        matrix[pos] = {}
        for state in states:
            # every cell is a dictionary with two values
            matrix[pos][state] = {"prob": "some_probability", "from": "some_state"}
    return matrix

In [ ]:
from pprint import pprint

pprint(matrix(["police", "police", "police"], ["N", "V"]), width=1)

Of course the code so far doesn't get us any closer to actually determining these values.
As the next step, let's expand the `matrix` function to a skeleton for the Viterbi algorithm.
We will intersperse the construction of the matrix with steps to fill each value with appropriate values.
However, we will still leave largely open how these values are computed.
The only thing we know is that they can only depend on the factors that are considered by the Viterbi algorithm:

1. the symbol at the current position of the input, and
1. the state, and
1. the preceding column.

In [ ]:
def viterbi(hmm, sentence):
    matrix = {}
    for pos in range(len(sentence)):
        matrix[pos] = {}
        # store previous column for later use ({} if non-existent)
        previous_column = matrix.get(pos-1, {})
        for state in states:
            # initialize empty cell
            matrix[pos][state] = {}
            # use cell as a shorthand
            cell = matrix[pos][state]
            # set values for cell
            cell["prob"], cell["from"] = some_mystery_function(hmm, sentence[pos], state, previous_column)
    return matrix

Believe it or not, this is actually the Viterbi algorithm in a nutshell.
Well, almost.
Of course we still have to figure out what the mystery function is.
Whatever it is, we want the function to return a pair of values.
The first one will be stored under the key `"prob"`, the second one under the key `"from"`.

## Finding the optimal cell in the previous row

The mystery function has the job of computing the probability value for the cell and the appropriate backpointer.
The Viterbi algorithm does this as follows for our example HMM:

1. Suppose we are in column `3` and state `N`.
1. The previous column is column `2`.
1. Each cell in column `2` belongs to a specific state, `V` or `N`.
1. For each state, we look up the probability of transitioning to `N`.
   From `N` to `N` it is `.3`, from `V` to `N` it is `.9`.
1. The transition probability is multiplied with the probability in the corresponding cell of the previous row.
   So if column 2 is `{'N': {"prob": .2, "from": 'V}, 'V': {"prob": .35, "from": 'N'}`, then the relevant multiplications are
   - from `N` to `N`: `.3 * .2`
   - from `V` to `N`: `.9 * .35`.
1. The probability for the current cell is based on the larger one of the two, i.e. `.9 * .35`, which is `.315`.
   We then multiply this value with the emission probability.
1. Since this probability is based on the values for `V`, the backpointer is set to `V`.

In sum, we have to iterate over all the cells in the previous column and check which one gives us the highest probability.
We then compute the overall probability based on that and the emission probability.
Let us focus only on the first step for now: finding the optimal cell in the previous row.

There's many ways of doing this.
We'll look at two different options here.
For both of them, it will be useful to have a helper function to quickly get the relevant transition probability in a safe manner.

In [ ]:
def tprob(hmm, source, goal):
    return hmm.trans.get(source, {}).get(goal, 0)

This function looks up the probability for going from state `source` to state `goal` in some HMM.
If for some reason this probability cannot be found, the function returns 0 as a safe default.
Simple enough, so let's move on to the first implementation for calculating the cell value.

In [ ]:
def indexed_max(hmm, state, matrix_column):
    # inititalize backpointer and highest probability
    backpointer = None
    optimal = 0
    # iterate over cells in column
    for cell_state, cell_values in matrix_column.items():
        # reminder: each column is a dictionary of the form
        # {state: {"prob": some_value, "from": some_vale}}
        cell_state_prob = cell_values["prob"]
        # prob = cell probability * transition probability
        prob = cell_state_prob * tprob(hmm, cell_state, state)
        # if we found something more likely, update optimal and backpointer
        if prob > optimal:
            optimal = prob
            backpointer = cell_state
    # loop over: optimal and backpointer are set to best possible values
    # we return a pair
    return (optimal, backpointer)

The `indexed_max` function iterates over all cells of the previous column and keeps updating the values for `optimal` and `backpointer` whenever it comes across a cell that offers a higher probability value than whatever is currently stored in `optimal`.

With our example above, Python would go through the following steps.

In [ ]:
# state is 'N'
# column 2 is {'N': {"prob": .2, "from": 'V'},
#              'V': {"prob": .35, "from": 'N'}

# initialization
backpointer = None
optimal = 0

# start for-loop
cell_state = 'V'
cell_values = {"prob:" .35, "from": 'N'}
cell_state_prob = .35
prob = .35 * tprob(hmm, 'N', 'N')  # .35 * .9

# new values are bigger
backpointer = 'V'
optimal = 0.315

# continue for-loop with next cell
cell_state = 'N'
cell_values = {"prob:" .2, "from": 'V'}
cell_state_prob = .2
prob = .2 * tprob(hmm, 'N', 'N')  # .2 * .3

# new values aren't bigger, keep backpointer and optimal the same

# for-loop done, return (optimal, backpointer)

Remember that `indexed_max` doesn't return the final probability for the cell yet at it does not factor in emission probabilities.
This will have to be handled by a different function later on.

Even though `indexed_max` is pretty simple, it may seem a little cluttered.
Alternatively, we could use list comprehensions and Python's `max` function.

In [ ]:
def list_max(hmm, state, matrix_column):
    options = [(cell_state,
                cell_values["prob"] * tprob(hmm, cell_state, state))
               for cell_state, cell_values in matrix_column.items()]
    return max(options, key=lambda x: x[1])

This function first constructs a list of `(state, probability)`-pairs and then uses `max` to extract the pair with the highest probability.
The code `key=lambda x: x[1]` tells `max` to only pay attention to the second component of each pair.

Python proceeds quite a bit differently with this function.

In [ ]:
# state is 'N'
# column 2 is {'N': {"prob": .2, "from": 'V'},
#              'V': {"prob": .35, "from": 'N'}

# construct list
options = [('N', 0.06), ('V', 0.315)]

# return max based on second component of pairs
return ('V', 0.315)

This looks like it might be more efficient since it involves fewer steps.
But in facts, this is less efficient because it requires the construction of a list.
Whereas `indexed_max` only iterates over existing data structures and stores small intermediate values in two variables, `list_max` has to construct a new list with as many items as there are states.
As HMM can easily contain thousands of states, constructing this list from scratch for every new cell is very wasteful.
Hence we will use `indexed_max` in this notebook even though it isn't as elegant as `list_max`.

## Adding emission probabilities

With `indexed_max` we only get a backpointer to a cell and the basic probability obtained by multiplying that cell's probability with the transition probability.
We still need to factor in the transmission probability.
That is easily done with a separate function `cell_value` that calls `indexed_max`.

In [ ]:
def cell_value(hmm, word, state, matrix_column):
    optimal, backpointer = indexed_max(hmm, state, matrix_column)
    optimal *= hmm.emissions.get(state, {}).get(word, 0)
    return (optimal, backpointer)

This function first gets the values for `optimal` and `backpointer` from `indexed_max`, and then multiplies `optimal` by the emission probability for the current word and state according to the HMM.
If there is no emission probabilty, `0` is used as a default.

All we need to do now is to got back to the definition of the `viterbi` function and replace `some_mystery_function` with `cell_value`.

In [ ]:
def viterbi(hmm, sentence):
    matrix = {}
    for pos in range(len(sentence)):
        matrix[pos] = {}
        # store previous column for later use ({} if non-existent)
        previous_column = matrix.get(pos-1, {})
        for state in states:
            # initialize empty cell
            matrix[pos][state] = {}
            # use cell as a shorthand
            cell = matrix[pos][state]
            # set values for cell
            cell["prob"], cell["from"] = cell_value(hmm, sentence[pos], state, previous_column)
    return matrix

Alas, this won't quite work because of a major oversight on our part.
The very first column of the table doesn't have a preceding column.
So `previous_column` will be set to `{}`.
Now consider what happens when Python runs `cell_value(hmm, sentence[0], state, {})`.
According to the definition of `cell_value`, we immediately call `indexed_max(hmm, state, {})`.
This causes `indexed_max` to return `(0, None)`.
That's because `{}` contains no items to iterate over, so we never get to overwrite the default values of `optimal = 0` and `backpointer = None`.
As every column depends on the values of the previous column, screwing up the first column means screwing up the whole table!

In order to fix this, we have to tell `cell_value` that it should computed the values of the first column differently.
For the first column, the backpointer should always refer to the HMM's initial state, and the probability is just the transition probability from the initial state.

In [ ]:
def cell_value(hmm, word, state, matrix_column):
    if not matrix_column:
        # we're working on the first column!
        optimal = tprob(hmm.start, state)
        backpointer = hmm.start
    else:
        optimal, backpointer = indexed_max(hmm, state, matrix_column)
    optimal *= hmm.emissions.get(state, {}).get(word, 0)
    return (optimal, backpointer)

With this modified function, we finally have a working implementation of the Viterbi algorithm.

## Converting to class methods

We can now add all the functions defined above as methods to the `hmm` class.
Since many of these functions are just helper methods that aren't meant to be directly used by the user, we add an underscore `_` in front of their name.
This is just a naming convention and does not change at all how they're treated by Python.
Also keep in mind that we have to add `self` as the first argument to each function.
Often this replaces the argument `hmm` that we used in the function definitions above.
This also means that code like `hmm.emissions` is replaced by `self.emissions`.

In [ ]:
class hmm():
    def __init__(self, start=0, final=-1, trans=[], emissions=[]):
        self.start = start
        self.final = -1
        self.states = set()
        self.trans = {}
        for t in trans:
            self.update(t)
        if isinstance(emissions, dict):
            self.emissions = emissions
        else:
            self.emissions = {}
            for e in emissions:
                self.set_emit(e)
        for state in self.states:
            self.trans[state] = self.trans.get(state, {})
            self.trans[state][final] = 1
        
        
    def update(self, trans):
        source, goal, prob = trans
        self.trans[source] = self.trans.get(source, {})
        self.trans[source][goal] = prob
        # add new states to hmm's set of states
        if source != self.start:
            self.states.add(source)
        if goal != self.final:
            self.states.add(goal)
        
        
    def set_emit(self, e):
        state, word, prob = e
        self.emissions[state] = self.emissions.get(state, {})
        self.emissions[state][word] = prob
        
        
    def _prob(self, source, goal):
        return self.trans.get(source, {}).get(goal, 0)
    
    
    def _indexed_max(self, state, matrix_column):
        backpointer = None
        optimal = 0
        for cell_state, cell_values in matrix_column.items():
            cell_state_prob = cell_values["prob"]
            prob = cell_state_prob * self._prob(cell_state, state)
            if prob > optimal:
                optimal = prob
                backpointer = cell_state
        return (optimal, backpointer)
            
    
    def _cell_value(self, word, state, matrix_column):
        if not matrix_column:
            # no matrix_column exists, use default values for column 1
            optimal = self._prob(self.start, state)
            backpointer = self.start
        else:
            optimal, backpointer = self._indexed_max(state, matrix_column)
        optimal *= self.emissions.get(state, {}).get(word, 0)
        return (optimal, backpointer)
        
        
    def viterbi(self, sentence):
        # compute matrix column by column;
        matrix = dict()
        for pos in range(len(sentence)):
            # start new column
            matrix[pos] = dict()
            # and get previous column, if it exists
            previous_column = matrix.get(pos-1, dict())
            # for each column, fill in the state rows
            for state in self.states:
                # create empty cell
                matrix[pos][state] = dict()
                cell = matrix[pos][state]
                # and compute its probability and backpointer
                cell["prob"], cell["from"] = self._cell_value(sentence[pos], state, previous_column)
        return matrix

We can finally see our code in action.

In [ ]:
from pprint import pprint

police = hmm(trans=[(0, "N", .8),
                    (0, "V", .2),
                    ("N", "N", .3),
                    ("N", "V", .7),
                    ("V", "N", .9),
                    ("V", "V", .1)],
             emissions=[("V", "police", 1),
                        ("N", "police", 1)])

# test sentence: [police [that the police does police]] does police the police]
ambiguous = ["police"] * 5
print(ambiguous)

pprint(police.viterbi(ambiguous), width=1)

Alright, that's the kind of table we need for the Viterbi algorithm.
But its still not the actual output.
We want to know the most likely state assignment for the sentence.
For this, we have to add another method.

# Finding the best path

Given a Viterbi table, we can find the best path in two steps:

1. In the last column, find the cell with the highest probability value.
1. Follow the backpointers all the way to the start.

The function below does just that, but in a somewhat clever fashion.
Pay close attention to the use of `indexed_max`, and try to figure out how it picks out the optimal cell in the last row.

In [ ]:
def bestpath(hmm, matrix, sentence):
    length = len(sentence)
    pick = hmm._indexed_max(hmm.final, matrix[length - 1])[1]
    path = [pick]
    for pos in reversed(range(1, length)):
        pick = matrix[pos][pick]["from"]
        path.append(pick)
    return path

Got it?
No?
Alright, let's go through it:

1. `matrix[length - 1]` gives us the last column of the Viterbi table.
1. Remember that every state has a transition to the final state, with a probability of one.
1. Usually, `hmm._indexed_max(hmm.final, matrix[length - 1])` would pick the best cell based on its stored probability and the transition probability.
   But since all states have the same transition probability of 1 to the final state, only the stored probability matters.
1. Consequently, `hmm._indexed_max` returns the cell in the last row with the highest probability value.

Once that cell has been found, we make it the first element of our variable `path`.
At this point, we only have to follow the backpointers.
To do this, we have to look up the value of the backpointer in the last but one column, get the backpointer there and check the corresponding cell in the last but two column, and so on.
This is a simple for loop with `range`, except that we use `reversed`:

1. `range(1, length)` goes from `1` to whatever the value of `length` is.
1. `reversed` switches the ordering; for instance, (1, 2, 3, 4) becomes (4, 3, 2, 1).

So we start with the last column, look at the value of `pick` there (i.e. the cell with the highest probability), and get the backpointer stored under the key `"from"`.
This is the new value for `pick`.
It gets added to `path`, then we look at the last but one column, check the cell indicated by the backpointer there, get its backpointer via `"from"`, make that the new pick, add it to path, move on to the column before that, and so on.
Notice that we do not check the very first column (`range` starts at `1`) because all the backpointers there always point to the initial state, which is of no interest to us.

We modify `bestpath` so that it intersperses the elements of path with the elements of the input, giving us a nice annotation of the whole input.
Keep in mind that `path` starts with the last state and then proceeds towards the first, so we have to reverse its order when interspersing it with the input.

In [ ]:
def bestpath(hmm, matrix, sentence):
    length = len(sentence)
    pick = hmm._indexed_max(hmm.final, matrix[length - 1])[1]
    path = [pick]
    for pos in reversed(range(1, length)):
        pick = matrix[pos][pick]["from"]
        path.append(pick)
    return [(sentence[pos], path[-(pos+1)]) for pos in range(length)]

One final modification: if at any point `pick` is set to `None`, we have run into a pathological case where, for some reason, no optimal path exists at all.
This can happen if the input contains a symbol for which every state has an emission probability of `0`.
Since there is no optimal path, we should return an empty list instead.

In [ ]:
def bestpath(hmm, matrix, sentence):
    length = len(sentence)
    pick = hmm._indexed_max(hmm.final, matrix[length - 1])[1]
    path = [pick]
    for pos in reversed(range(1, length)):
        if pick is None:
            return []
        pick = matrix[pos][pick]["from"]
        path.append(pick)
    return [(sentence[pos], path[-(pos+1)]) for pos in range(length)]

## Putting it all together: the `hmm` class

The `bestpath` function is our last addition to the `hmm` class.
As a convenient shorthand, we define the method `parse` that first constructs a Viterbi table and then constructs the best path from that.
We also add docstrings to all our methods.

In [ ]:
class hmm():
    def __init__(self, start=0, final=-1, trans=[], emissions=[]):
        """Class for Hidden Markov models.
        
        Arguments
        ---------
        start: int or string
            name of the unique initial state
            default: 0
        final: int or string
            name of the unique final state
            default: -1
        transitions: list
            list of transition tuples of the form (source, goal, probability)
            default: empty list
        emissions: list
            list of emission tuples of the form (state, word, probability)
            
        Other attributes
        ----------------
        trans: dict
            dictionary of transitions; use the format {source: {goal: probability}}
        states: set
            set of states, excluding initial and final
        """
        self.start = start
        self.final = -1
        self.states = set()
        self.trans = {}
        for t in trans:
            self.update(t)
        if isinstance(emissions, dict):
            self.emissions = emissions
        else:
            self.emissions = {}
            for e in emissions:
                self.set_emit(e)
        for state in self.states:
            self.trans[state] = self.trans.get(state, {})
            self.trans[state][final] = 1
        
        
    def update(self, trans):
        """Convert transmission triple to entry in transition dictionary."""
        source, goal, prob = trans
        self.trans[source] = self.trans.get(source, {})
        self.trans[source][goal] = prob
        # add new states to hmm's set of states
        if source != self.start:
            self.states.add(source)
        if goal != self.final:
            self.states.add(goal)
        
        
    def set_emit(self, e):
        """Convert emission triple to entry in emission dictionary."""
        state, word, prob = e
        self.emissions[state] = self.emissions.get(state, {})
        self.emissions[state][word] = prob
        
        
    def _prob(self, source, goal):
        """Retrieve transition probability from source to goal."""
        return self.trans.get(source, {}).get(goal, 0)
    
    
    def _indexed_max(self, state, matrix_column):
        """Find highest probability value and the cell it is derived from."""
        backpointer = None
        optimal = 0
        for cell_state, cell_values in matrix_column.items():
            cell_state_prob = cell_values["prob"]
            prob = cell_state_prob * self._prob(cell_state, state)
            if prob > optimal:
                optimal = prob
                backpointer = cell_state
        return (optimal, backpointer)
            
    
    def _cell_value(self, word, state, matrix_column):
        """Compute the value of a Viterbi cell."""
        if not matrix_column:
            # no matrix_column exists, use default values for column 1
            optimal = self._prob(self.start, state)
            backpointer = self.start
        else:
            optimal, backpointer = self._indexed_max(state, matrix_column)
        optimal *= self.emissions.get(state, {}).get(word, 0)
        return (optimal, backpointer)
        
        
    def viterbi(self, sentence):
        """Construct a Viterbi table with all cells filled."""
        # compute matrix column by column;
        matrix = dict()
        for pos in range(len(sentence)):
            # start new column
            matrix[pos] = dict()
            # and get previous column, if it exists
            previous_column = matrix.get(pos-1, dict())
            # for each column, fill in the state rows
            for state in self.states:
                # create empty cell
                matrix[pos][state] = dict()
                cell = matrix[pos][state]
                # and compute its probability and backpointer
                cell["prob"], cell["from"] = self._cell_value(sentence[pos], state, previous_column)
        return matrix
    
    
    def _bestpath(self, matrix, sentence):
        """Find the best path through a Viterbi table."""
        length = len(sentence)
        pick = self._indexed_max(self.final, matrix[length - 1])[1]
        path = [pick]
        for pos in reversed(range(1, length)):
            if pick is None:
                return []
            pick = matrix[pos][pick]["from"]
            path.append(pick)
        return [(sentence[pos], path[-(pos + 1)]) for pos in range(length)]
    
    
    def parse(self, sentence):
        """Annotate the input with the most likely state assignment."""
        matrix = self.viterbi(sentence)
        return self._bestpath(matrix, sentence)

In [ ]:
from pprint import pprint

police = hmm(trans=[(0, "N", .8),
                    (0, "V", .2),
                    ("N", "N", .3),
                    ("N", "V", .7),
                    ("V", "N", .9),
                    ("V", "V", .1)],
             emissions=[("V", "police", 1),
                        ("N", "police", 1)])

# test sentence: [police [that the police does police]] does police the police]
ambiguous = ["police"] * 5
print(ambiguous)
pprint(police.viterbi(ambiguous), width=1)
print(police.parse(ambiguous))

In [ ]:
# a ridiculously challenging example with 100,000 words;
# look how fast this still is!!!
super_ambiguous = ["police"] * (10 ** 5)
police.parse(super_ambiguous)
;  # hide output, it takes longer to print than to compute

As you can see, it's quite a bit of code.
You probably would have found this to be a very challenging homework assignment.
But by proceeding step by step, even complex algorithms can be broken done into simple functions/methods that are then combined to yield the desired result.

## Bullet point summary

- The Viterbi algorithm has three key components:
    1. a table where columns represent positions in the input and rows states in the automaton
    1. a mechanism for computing the optimal probability for a cell based on the cells in the preceding column
    1. a mechanism for computing the optimal path from the backpointers in the table
- The table can be implemented as a dictionary of the form

```python
{column_number: {state: {probability: some_value, backpointer: some_value}}}
```

- The hardest part is determining a cell's value.
  This requires iterating over all cells in the previous column to determine which one leads to the highest probability value.
  
- Once the table is filled in, the optimal state assignment is found by picking the cell with highest probability in the last column and following the backpointers all the way to column 1.